In [1]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

import random


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device: " + DEVICE)
if torch.backends.cudnn.is_available():
    torch.backends.cudnn.enabled = True

SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

Using device: cuda


## Preparing data

In [2]:
def read_data_with_norm(file_path_X, file_path_y, seq_length):
    X_data = np.array(pd.read_csv(file_path_X, header=None))
    y_data = np.array(pd.read_csv(file_path_y, header=None))
    
    # Shift labels in PyTorch classification models labels start from 0
    y_data = y_data - 1
    
    blocks = X_data.shape[0] / seq_length
    
    X_seq = np.array(np.split(X_data, blocks, axis=0))
    
    return X_seq, y_data

In [3]:
def read_data(file_path_X, file_path_y, seq_length):
    X_data = np.array(pd.read_csv(file_path_X, sep="\t", header=None, dtype=np.float32))
    y_data = np.array(pd.read_csv(file_path_y, sep="\t", header=None, dtype=np.int_))
    
    blocks = X_data.shape[0] / seq_length
    
    X_seq = np.array(np.split(X_data, blocks, axis=0))
    
    return X_seq, y_data

In [4]:
SEQ_LENGTH = 16

X_all, y_all = read_data("data/shortened_shuffled_poses.csv","data/shortened_shuffled_labels.csv", SEQ_LENGTH)

In [5]:
val_size = int(y_all.shape[0] * 0.2)
val_size

4439

In [6]:
X_val, y_val = X_all[-val_size:], y_all[-val_size:]

In [7]:
X_train, y_train = X_all[:-val_size], y_all[:-val_size]

In [8]:
y_val.shape

(4439, 1)

In [9]:
X_train.shape[2]

20

In [10]:
train_dataset = torch.utils.data.TensorDataset(torch.tensor(X_train, dtype=torch.float32),
                                               torch.tensor(y_train, dtype=torch.long).squeeze())

val_dataset = torch.utils.data.TensorDataset(torch.tensor(X_val, dtype=torch.float32),
                                             torch.tensor(y_val, dtype=torch.long).squeeze())

## NN model

In [11]:
class LstmClassifier(nn.Module):
    
    def __init__(self, input_dim, n_classes, lstm_hidden_dim=256, fc_hidden_dim=256, n_lstm_layers=2):
        super(LstmClassifier, self).__init__()
        
        self._lstm = nn.LSTM(input_size=input_dim,
                             hidden_size=lstm_hidden_dim,
                             num_layers=n_lstm_layers,
                             batch_first=True)
        
        self._fc = nn.Sequential(nn.Linear(lstm_hidden_dim, fc_hidden_dim),
                                 nn.ReLU(),
                                 nn.Linear(fc_hidden_dim, n_classes))
        
    def forward(self, x):
        lstm_output, _ = self._lstm.forward(x)
        lstm_output = lstm_output[:, -1, :]
        fc_output = self._fc.forward(lstm_output)
        return fc_output

## Training utils

In [12]:
def run_epoch(model, optimizer, criterion, batches, phase='train'):
    is_train = phase == 'train'
    if is_train:
        model.train()
    else:
        model.eval()

    epoch_loss = 0.0
    n_predictions = 0
    
    correct_predictions = 0

    for X_batch, y_batch in batches:
        X_batch = X_batch.to(DEVICE)
        y_batch = y_batch.to(DEVICE)

        with torch.set_grad_enabled(is_train):
            y_pred = model.forward(X_batch)
            loss = criterion.forward(y_pred, y_batch)
    
        if is_train:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        epoch_loss += loss.item() * y_batch.shape[0]
        correct_predictions += (torch.argmax(y_pred, dim=1) == y_batch).sum().item()
        n_predictions += y_batch.shape[0]

    epoch_loss = epoch_loss / n_predictions
    epoch_accuracy = correct_predictions / n_predictions

    return epoch_loss, epoch_accuracy


def train_model(model, optimizer, criterion, n_epoch, batch_size, train_dataset, val_dataset, backup_name):
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    best_val_loss = np.inf
    
    train_batches = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=4, shuffle=True)
    val_batches = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, num_workers=4, shuffle=False)

    for epoch in range(n_epoch):
        train_loss, train_accuracy = run_epoch(model, optimizer, criterion, train_batches, phase='train')
        val_loss, val_accuracy = run_epoch(model, optimizer, criterion, val_batches, phase='val')

        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), backup_name)

        print("Epoch: " + str(epoch))
        print("Train loss: " + str(train_loss) + ", accuracy: " + str(train_accuracy))
        print("Val loss: " + str(val_loss) + ", accuracy: " + str(val_accuracy) + "\n\n")
        
    return train_losses, train_accuracies, val_losses, val_accuracies

## Training process

In [15]:
N_CLASSES = 8
INPUT_DIM = X_train.shape[2]

In [16]:

model = LstmClassifier(INPUT_DIM, N_CLASSES)
model = model.to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

train_losses, train_accuracies, val_losses, val_accuracies= train_model(model,
            optimizer=torch.optim.Adam(model.parameters(), lr=1e-3),
            criterion=nn.CrossEntropyLoss(),
            n_epoch=500,
            batch_size=500,
            train_dataset=train_dataset,
            val_dataset=val_dataset,
            backup_name="lstm_action_classifier.pth.tar")

ss: 0.037281900703876804, accuracy: 0.9877808435159637
Val loss: 0.4594656147581415, accuracy: 0.9252083802658256


Epoch: 352
Train loss: 0.049620688715488685, accuracy: 0.9836702517033616
Val loss: 0.5004106191974901, accuracy: 0.9105654426672674


Epoch: 353
Train loss: 0.04001650474560718, accuracy: 0.9859226307787601
Val loss: 0.4658857426629407, accuracy: 0.9114665465194863


Epoch: 354
Train loss: 0.09201532098234919, accuracy: 0.9699870488203165
Val loss: 0.5998846716938805, accuracy: 0.879477359765713


Epoch: 355
Train loss: 0.10683405222864703, accuracy: 0.9644124106087054
Val loss: 0.4805608064399482, accuracy: 0.8995269204775851


Epoch: 356
Train loss: 0.10943778307571544, accuracy: 0.9629483642096965
Val loss: 0.47357225103216094, accuracy: 0.9026807839603515


Epoch: 357
Train loss: 0.07333361695388582, accuracy: 0.9740413311560335
Val loss: 0.44804490424942717, accuracy: 0.9123676503717053


Epoch: 358
Train loss: 0.0391337609535164, accuracy: 0.9862041781631848
Val lo

In [22]:
training = pd.DataFrame({"train_losses":train_losses, "train_accuracies":train_accuracies, "val_losses":val_losses, "val_accuracies":val_accuracies})


In [24]:
training.to_csv("training_vals",  sep="\t", index=False)

## Load and Inference

In [7]:
model = LstmClassifier(INPUT_DIM, N_CLASSES)
model = model.to(DEVICE)
model.load_state_dict(torch.load("lstm_action_classifier.pth.tar"))

<All keys matched successfully>

In [8]:
def run_inference_on_sequence(model, sequence):
    model.eval()
    X_batch = sequence.to(DEVICE)

    with torch.no_grad():
        y_pred = model.forward(X_batch)
        print(y_pred)
    return torch.argmax(y_pred, dim=1)

In [29]:
sequence = train_dataset[45][0].unsqueeze(0)
predictions = run_inference_on_sequence(model, sequence)
print(predictions.item())

tensor([[ 7.6807, -7.4713, -1.3673, -8.8626, -7.3632, -1.4662]],
       device='cuda:0')
0


In [25]:
predictions = run_inference_on_sequence(model, sequence)
print(predictions.item())

tensor([[ 6.3528, -7.8725, -2.6291, -6.6467, -6.0407, -5.5711]],
       device='cuda:0')
0


0
